In [85]:
from dotenv import load_dotenv

load_dotenv() # load session

True

In [86]:
import re
from functools import reduce
from itertools import groupby
from aocd import get_data, submit
day = 7
year = 2023

input = get_data(day=day, year=year)

# submit(answer=result, part="a", day=day, year=year)

In [87]:
text = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

In [146]:
from functools import cmp_to_key

labels = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
hand_types = ['high_card', 'one_pair', 'two_pair', 'three_of_a_kind', 'full_house', 'four_of_a_kind', 'five_of_a_kind']

def compare_card(card1, card2):
  return labels.index(card1) - labels.index(card2)

assert compare_card('2', '3') < 0
assert compare_card('A', '3') > 0
assert compare_card('K', 'K') == 0

def evaluate_hand(hand):
  def rest(cards):
    hands_minus_cards = set(hand) - set(cards)
    return sorted(list(hands_minus_cards), key=cmp_to_key(compare_card), reverse=True)

  cards = {label: len(list(cards))
          for label, cards in groupby(sorted(hand, key=cmp_to_key(compare_card), reverse=True))}
  
  for label, number in sorted(cards.items(), key=lambda item: item[1], reverse=True):
    if number == 5:
      return ('five_of_a_kind', [label])
    elif number == 4:
      return ('four_of_a_kind', [label] + rest(label))
    elif number == 3:
      if len(set([label for label in hand])) == 2:
        return ('full_house', [label] + rest(label))
      else:
        return ('three_of_a_kind', [label] + rest(label))
    elif number == 2:
      if len(set([label for label in hand])) == 3:
        other_label = [label2 for label2 in hand if label != label2 and cards[label2] == 2][0]
        return ('two_pair', [label, other_label] + rest([label, other_label]))
      else:
        return ('one_pair', [label] + rest(label))
    else:
      return ('high_card', rest([]))

assert evaluate_hand("AAAAA") == ('five_of_a_kind', ['A'])
assert evaluate_hand("AAAAQ") == ('four_of_a_kind', ['A', 'Q'])
assert evaluate_hand("QAAAA") == ('four_of_a_kind', ['A', 'Q'])
assert evaluate_hand("AAAQQ") == ('full_house', ['A', 'Q'])
assert evaluate_hand("QQQAA") == ('full_house', ['Q', 'A'])
assert evaluate_hand("QQQAK") == ('three_of_a_kind', ['Q', 'A', 'K'])
assert evaluate_hand("QQQKA") == ('three_of_a_kind', ['Q', 'A', 'K'])
assert evaluate_hand("QQQ23") == ('three_of_a_kind', ['Q', '3', '2'])
assert evaluate_hand("QQQ32") == ('three_of_a_kind', ['Q', '3', '2'])
assert evaluate_hand("44332") == ('two_pair', ['4', '3', '2'])
assert evaluate_hand("44223") == ('two_pair', ['4', '2', '3'])
assert evaluate_hand("44AQJ") == ('one_pair', ['4', 'A', 'Q', 'J'])
assert evaluate_hand("44QAJ") == ('one_pair', ['4', 'A', 'Q', 'J'])
assert evaluate_hand("44JQA") == ('one_pair', ['4', 'A', 'Q', 'J'])
assert evaluate_hand("23456") == ('high_card', ['6', '5', '4', '3', '2'])
assert evaluate_hand("63542") == ('high_card', ['6', '5', '4', '3', '2'])

In [147]:
def best_hand(cards1, cards2): # poker version
  for i, _ in enumerate(cards1):
    comp = compare_card(cards1[i], cards2[i])
    if comp != 0:
      return comp
  return 0

def best_hand2(hand1, hand2): # camel card version
  for i, _ in enumerate(hand1):
    comp = compare_card(hand1[i], hand2[i])
    if comp != 0:
      return comp
  return 0

def compare_hands(hand1, hand2):
  type1, cards1 = evaluate_hand(hand1)
  type2, cards2 = evaluate_hand(hand2)
  
  if type1 == type2:
    return best_hand2(hand1, hand2)
  else:
    return hand_types.index(type1) - hand_types.index(type2)
  
assert compare_hands("33332", "2AAAA") > 0
assert compare_hands("77888", "77788") > 0
assert compare_hands("AAAAA", "A2345") > 0
assert compare_hands("A2345", "AAAAA") < 0
assert compare_hands("AAAAQ", "AAQQK") > 0
assert compare_hands("22334", "QQA23") > 0
assert compare_hands("22243", "2345A") > 0
assert compare_hands("AAAAA", "KKKKK") > 0
assert compare_hands("QQQQ2", "JJJJ3") > 0
assert compare_hands("QQQ22", "JJJ33") > 0
assert compare_hands("QQQ35", "QQQ34") > 0
assert compare_hands("4499A", "44992") > 0
assert compare_hands("44236", "44235") > 0
assert compare_hands("23457", "23456") > 0

In [132]:
# print(text)

def read_line(line):
  parts = line.split(" ")
  return (parts[0], int(parts[1]))

def read_hands_and_bids(text):
  return [read_line(line) for line in text.splitlines()]

def compare_hand_and_bid(hand1, hand2):
  return compare_hands(hand1[0], hand2[0])

example_hands_and_bids = read_hands_and_bids(text)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(example_hands_and_bids, key=cmp_to_key(compare_hand_and_bid)), start=1)])

assert result == 6440

In [133]:
text2 = """2345A 1
Q2KJJ 13
Q2Q2Q 19
T3T3J 17
T3Q33 11
2345J 3
J345A 2
32T3K 5
T55J5 29
KK677 7
KTJJT 34
QQQJA 31
JJJJJ 37
JAAAA 43
AAAAJ 59
AAAAA 61
2AAAA 23
2JJJJ 53
JJJJ2 41"""

example_hands_and_bids2 = read_hands_and_bids(text2)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(example_hands_and_bids2, key=cmp_to_key(compare_hand_and_bid)), start=1)])

assert result == 6592


In [134]:
hands_and_bids = read_hands_and_bids(input)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(hands_and_bids, key=cmp_to_key(compare_hand_and_bid)), start=1)])

submit(answer=result, part="a", day=day, year=year)

aocd will not submit that answer again. At 2023-12-07 14:41:04.806291-05:00 you've previously submitted 256448566 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [172]:
labels = ['J', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'Q', 'K', 'A']
hand_types = ['high_card', 'one_pair', 'two_pair', 'three_of_a_kind', 'full_house', 'four_of_a_kind', 'five_of_a_kind']

def compare_card(card1, card2):
  return labels.index(card1) - labels.index(card2)

assert compare_card('2', 'J') > 0
assert compare_card('2', '3') < 0
assert compare_card('A', '3') > 0
assert compare_card('K', 'K') == 0

In [187]:
def evaluate_hand(hand):
  joker_in_hand = 'J' in hand

  cards = {label: len(list(cards))
          for label, cards in groupby(sorted(hand, key=cmp_to_key(compare_card), reverse=True))}
  
  for label, number in sorted(cards.items(), key=lambda item: item[1], reverse=True):
    if number == 5:
      return 'five_of_a_kind'
    elif number == 4:
      if not joker_in_hand:
        return 'four_of_a_kind'
      else: 
        # JJJJA returns five_of_a_kind
        # AAAAJ returns five_of_a_kind
        return 'five_of_a_kind'
    elif number == 3:
      is_full_house = len(set(hand)) == 2
      if is_full_house:
        if not joker_in_hand:
          return 'full_house'
        else:
          # JJJQQ returns five_of_a_kind
          # QQQJJ returns five_of_a_kind
          return 'five_of_a_kind'
      else: # not full house
        if not joker_in_hand:
          return 'three_of_a_kind'
        else:
          # JJJ23 returns four_of_a_kind
          # QQQJ2 returns four_of_a_kind
          return 'four_of_a_kind'
    elif number == 2:
      is_two_pair = len(set(hand)) == 3
      if is_two_pair:
        other_label = [label2 for label2 in hand if label != label2 and cards[label2] == 2][0]
        if not joker_in_hand:
          return 'two_pair'
        else:
          if label == 'J' or other_label == 'J':
            # JJQQ2 returns four_of_a_kind
            # QJJJ2 returns four_of_a_kind
            return 'four_of_a_kind'
          else: # J is the last label
            # QQAAJ returns full_house
            return 'full_house'
      else: # not two pair
        if not joker_in_hand:
          return 'one_pair'
        else:
          # JJQ23 returns three_of_a_kind
          # AAJQ3 returns three_of_a_kind
          return 'three_of_a_kind'
    else:
      if not joker_in_hand:
        return 'high_card'
      else:
        return 'one_pair'

assert evaluate_hand("AAAAA") == 'five_of_a_kind'
assert evaluate_hand("AAAA2") == 'four_of_a_kind'
assert evaluate_hand("AAA22") == 'full_house'
assert evaluate_hand("AAA23") == 'three_of_a_kind'
assert evaluate_hand("AA223") == 'two_pair'
assert evaluate_hand("AA823") == 'one_pair'
assert evaluate_hand("A9823") == 'high_card'
assert evaluate_hand("JAAAA") == 'five_of_a_kind'
assert evaluate_hand("JAAA2") == 'four_of_a_kind'
assert evaluate_hand("JAA22") == 'full_house'
assert evaluate_hand("JAA23") == 'three_of_a_kind'
assert evaluate_hand("JA223") == 'three_of_a_kind'
assert evaluate_hand("AA823") == 'one_pair'
assert evaluate_hand("A9823") == 'high_card'

assert evaluate_hand("2345A") == "high_card"
assert evaluate_hand("J345A") == "one_pair"
assert evaluate_hand("2345J") == "one_pair"
assert evaluate_hand("32T3K") == "one_pair"
assert evaluate_hand("KK677") == "two_pair"
assert evaluate_hand("T3Q33") == "three_of_a_kind"
assert evaluate_hand("Q2KJJ") == "three_of_a_kind"
assert evaluate_hand("T3T3J") == "full_house"
assert evaluate_hand("Q2Q2Q") == "full_house"
assert evaluate_hand("2AAAA") == "four_of_a_kind"
assert evaluate_hand("T55J5") == "four_of_a_kind"
assert evaluate_hand("QQQJA") == "four_of_a_kind"
assert evaluate_hand("KTJJT") == "four_of_a_kind"
assert evaluate_hand("JJJJJ") == "five_of_a_kind"
assert evaluate_hand("JJJJ2") == "five_of_a_kind"
assert evaluate_hand("JAAAA") == "five_of_a_kind"
assert evaluate_hand("2JJJJ") == "five_of_a_kind"
assert evaluate_hand("AAAAJ") == "five_of_a_kind"
assert evaluate_hand("AAAAA") == "five_of_a_kind"

In [188]:
def best_hand2(hand1, hand2): # camel card version
  for i, _ in enumerate(hand1):
    comp = compare_card(hand1[i], hand2[i])
    if comp != 0:
      return comp
  return 0

def compare_hands(hand1, hand2):
  type1 = evaluate_hand(hand1)
  type2 = evaluate_hand(hand2)
  
  if type1 == type2:
    return best_hand2(hand1, hand2)
  else:
    return hand_types.index(type1) - hand_types.index(type2)
  
# assert compare_hands("2345A", "J345A") < 0

In [189]:
def read_line(line):
  parts = line.split(" ")
  return (parts[0], int(parts[1]))

def read_hands_and_bids(text):
  return [read_line(line) for line in text.splitlines()]

def compare_hand_and_bid(hand1, hand2):
  return compare_hands(hand1[0], hand2[0])

example_hands_and_bids = read_hands_and_bids(text)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(example_hands_and_bids, key=cmp_to_key(compare_hand_and_bid)), start=1)])

assert result == 5905

In [191]:
example_hands_and_bids2 = read_hands_and_bids(text2)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(example_hands_and_bids2, key=cmp_to_key(compare_hand_and_bid)), start=1)])

print(result)

assert result == 6839

6839


In [192]:
hands_and_bids = read_hands_and_bids(input)

result = sum([index * bid
              for index, (_, bid) in enumerate(sorted(hands_and_bids, key=cmp_to_key(compare_hand_and_bid)), start=1)])

submit(answer=result, part="b", day=day, year=year)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
